In [1]:
!nvidia-smi

Mon Jan  9 10:34:03 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.67       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 3090    Off  | 00000000:65:00.0  On |                  N/A |
| 68%   54C    P8    53W / 370W |   3878MiB / 24265MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 3090    Off  | 00000000:B3:00.0 Off |                  N/A |
| 68%   

In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import tensorflow as tf
import pickle
import pandas as pd
import numpy as np
import warnings
from models import *
from evaluation import *
from load_data import *
from resnet3d import *

print(tf.__version__)
warnings.filterwarnings("ignore")

gpus = tf.config.list_physical_devices(device_type='GPU')
tf.config.set_visible_devices(devices=gpus[1], device_type='GPU')
tf.config.experimental.set_memory_growth(gpus[1], True)

2.5.0


In [3]:
seed = 2021
os.environ['PYTHONHASHSEED']=str(seed)
tf.random.set_seed(seed)
np.random.seed(seed)

# Evaluate the predictions

## 3D MRI

In [20]:
df = pd.read_csv('data_new_balanced.csv')
data_path = '../../../mnt/usb/kuopc/ADNI_B1/MPR__GradWarp__B1_Correction_crop/'

df = df.loc[df['Group'] != 'MCI']
df = df.loc[df['Split'] == 'test']

df['Group'] = df['Group'].replace(['CN', 'AD'], [0, 1])
df['Sex'] = df['Sex'].replace(['F', 'M'], [0, 1])
df['Age'] = np.where(df['Age'] <= 75, 0, 1)
df['Race'] = np.where(df['Race'] < 1, 0, 1)

In [7]:
model_name = '3D_CNN_AD_CN_age'
prediction_name = 'predictions/{i}_on_original'.format(i=model_name)
with open(prediction_name, "rb") as fp:
    y_preds = pickle.load(fp)
demo_test(y_preds, df['Age'].values)

model_name = '3D_CNN_AD_CN_age'
prediction_name = 'predictions/{i}_on_aug'.format(i=model_name)
with open(prediction_name, "rb") as fp:
    y_preds = pickle.load(fp)
demo_test(y_preds, df['Age'].values)

model_name = '3D_CNN_AD_CN_age_proposed'
prediction_name = 'predictions/{i}_on_original'.format(i=model_name)
with open(prediction_name, "rb") as fp:
    y_preds = pickle.load(fp)
demo_test(y_preds, df['Age'].values)

model_name = '3D_CNN_AD_CN_age_proposed'
prediction_name = 'predictions/{i}_on_aug'.format(i=model_name)
with open(prediction_name, "rb") as fp:
    y_preds = pickle.load(fp)
demo_test(y_preds, df['Age'].values)

Sample auc mean: 0.687
Samole auc std: 0.033
Sample auc CI: 0.074
Confidence interval for the score: [0.612 - 0.761]
Sample auc mean: 0.693
Samole auc std: 0.033
Sample auc CI: 0.076
Confidence interval for the score: [0.618 - 0.769]
Sample auc mean: 0.583
Samole auc std: 0.037
Sample auc CI: 0.084
Confidence interval for the score: [0.499 - 0.667]
Sample auc mean: 0.691
Samole auc std: 0.033
Sample auc CI: 0.075
Confidence interval for the score: [0.616 - 0.766]


In [10]:
def compute_result(y_preds, y_test, best_thresh, demo, result_name, group):
    group_type = {'race': [0, 1], 'gender': [0, 1], 'age': [0, 1]}
    dfs = []

    for j in group_type[group]:

        idx = demo == j

        dfs.append(test(y_preds[idx], y_test[idx], best_thresh))

    with open("results/{i}_{group}_results".format(i=result_name, group=group), "wb") as fp:
        pickle.dump(dfs, fp)

In [21]:
model_name = '3D_CNN_AD_CN_balanced_proposed'
result_name = '{i}_on_aug'.format(i=model_name)
prediction_name = 'predictions/{i}_on_aug'.format(i=model_name)
best_thresh = np.loadtxt('thresh/{i}_thresh.txt'.format(i=model_name))

with open(prediction_name, "rb") as fp:
    y_preds = pickle.load(fp)

compute_result(y_preds, df['Group'].values, best_thresh, df['Race'].values, result_name, 'race')
compute_result(y_preds, df['Group'].values, best_thresh, df['Sex'].values, result_name, 'gender')
compute_result(y_preds, df['Group'].values, best_thresh, df['Age'].values, result_name, 'age')